    - Variables = Teams
    - Variable Assignment = Group
    - Domain = Pot
    - Constraint = constraint one & two above

In [ ]:
import argv[1]
filePath = sys.argv[1]
with open(filePath, 'r') as openFile:
  fileData = openFile.readlines()
openFile.close()
cspInfo = [line.strip() for line in fileData]



In [ ]:
import collections
dNumOfGroups = int(cspInfo[0])
dNumOfPots = int(cspInfo[1])
dctPots = collections.defaultdict(list)
dctConfederations = collections.defaultdict(list)
count = 2
intPotNum = 1
for pot in range(dNumOfPots):
  dctPots[intPotNum] = cspInfo[count]
  count = count + 1
  intPotNum = intPotNum + 1
  
for confederation in range(6):
  splitLine = cspInfo[count].split(':')
  strConfederation = splitLine.pop(0)
  splitLine = str(splitLine)
  splitLine = splitLine.split(',')
  count = count + 1
  for word in splitLine:
    word = word.strip('[')
    word = word.strip(']')
    word = word.strip("'")
    dctConfederation[strConfederation.append(word)]